In [1]:
from models.SSD300 import SSD300
import tensorflow as tf
keras = tf.keras

In [2]:
SSD300_model = SSD300().getModel()

In [3]:
SSD300_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                     loss=keras.losses.categorical_crossentropy,
                     metrics=['accuracy'])

In [4]:
SSD300_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1_1 (Conv2D)             (None, 300, 300, 64)      1792      
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 300, 300, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 64)      0         
_________________________________________________________________
Conv2_1 (Conv2D)             (None, 150, 150, 128)     73856     
_________________________________________________________________
Conv2_2 (Conv2D)             (None, 150, 150, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 128)       0         
_________________________________________________________________
Conv3_1 (Conv2D)             (None, 75, 75, 256)      

In [5]:
test_input = tf.ones((10, 300, 300, 3), dtype=tf.dtypes.float32, name=None)
result = SSD300_model(test_input)
print(result[0])

tf.Tensor(
[0.1000019  0.0999954  0.10000236 0.1000016  0.09999725 0.10000128
 0.09999787 0.10000142 0.09999599 0.10000492], shape=(10,), dtype=float32)


In [6]:
# load dataset
(trainX, trainy), (testX, testy) = keras.datasets.cifar10.load_data()

In [7]:
from sklearn.preprocessing import OneHotEncoder
# get y label in one-hot encoding
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(trainy)
trainy = enc.transform(trainy).toarray()

In [8]:
from  skimage import transform
import numpy as np

# resize images
train_x = trainX[:1000]
train_y = trainy[:1000]
new_shape = (300,300,3)
train_x = np.asarray([transform.resize(image, new_shape) for image in train_x])

In [9]:
#train model
SSD300_model.fit(train_x, train_y, batch_size=32, epochs=1)

32/32 [==============================] - 25s 768ms/step - loss: 2.3035 - accuracy: 0.1030
